In [1]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd

## Accuracy

In [2]:
acc_dir = Path("output/bbh/accuracy")

files = list(acc_dir.glob("*.csv"))
files

[PosixPath('output/bbh/accuracy/flan-t5-large_cot_False_k_1.csv'),
 PosixPath('output/bbh/accuracy/flan-t5-base_cot_False_k_1.csv'),
 PosixPath('output/bbh/accuracy/flan-t5-xxl_cot_False_k_1.csv'),
 PosixPath('output/bbh/accuracy/flan-t5-large_cot_True_k_1.csv'),
 PosixPath('output/bbh/accuracy/flan-t5-small_cot_True_k_1.csv'),
 PosixPath('output/bbh/accuracy/flan-t5-xl_cot_False_k_1.csv'),
 PosixPath('output/bbh/accuracy/flan-t5-base_cot_True_k_1.csv'),
 PosixPath('output/bbh/accuracy/flan-t5-xxl_cot_True_k_1.csv'),
 PosixPath('output/bbh/accuracy/flan-t5-small_cot_False_k_1.csv'),
 PosixPath('output/bbh/accuracy/flan-t5-xl_cot_True_k_1.csv')]

In [12]:
df = pd.concat([pd.read_csv(f) for f in files]).drop(columns=['k'])
df.head()

,model_name,task_name,accuracy,cot
0,flan-t5-large,boolean_expressions,0.54000,False
1,flan-t5-large,causal_judgement,0.57754,False
2,flan-t5-large,date_understanding,0.24400,False
3,flan-t5-large,disambiguation_qa,0.65200,False
4,flan-t5-large,dyck_languages,0.00000,False


In [17]:
df.groupby(["model_name", "cot"]).mean().unstack()

accuracy          
cot               False      True
model_name                       
flan-t5-base   0.316745  0.253406
flan-t5-large  0.373640  0.311213
flan-t5-small  0.297036  0.111279
flan-t5-xl     0.413981  0.349619
flan-t5-xxl    0.458086  0.412731

![](./output/flan_paper_result.png)

Mostly match paper results. One exception, Small CoT is 8% lower than paper. Why? Hunch is my brittle, primitive regex answer extraction is not as good as paper's.

In [27]:
df = pd.concat([pd.read_csv(f) for f in acc_dir.glob('*xxl*.csv')]).drop(columns=['k'])
df.groupby(['task_name', 'cot']).mean().unstack().sort_values(by=('task_name'))

accuracy          
cot                                         False      True
task_name                                                  
boolean_expressions                      0.580000  0.632000
causal_judgement                         0.609626  0.556150
date_understanding                       0.552000  0.608000
disambiguation_qa                        0.676000  0.632000
dyck_languages                           0.012000  0.008000
formal_fallacies                         0.544000  0.560000
geometric_shapes                         0.200000  0.268000
hyperbaton                               0.740000  0.624000
logical_deduction_five_objects           0.536000  0.484000
logical_deduction_seven_objects          0.612000  0.520000
logical_deduction_three_objects          0.720000  0.636000
movie_recommendation                     0.612000  0.400000
multistep_arithmetic_two                 0.016000  0.004000
navigate                                 0.596000  0.580000
object_counting                          0.436000  0.388000
penguins_in_a_table                      0.431507  0.431507
reasoning_about_colored_objects          0.592000  0.500000
ruin_names                               0.504000  0.340000
salient_translation_error_detection      0.416000  0.328000
snarks                                   0.747191  0.528090
sports_understanding                     0.692000  0.632000
temporal_sequences                       0.344000  0.284000
tracking_shuffled_objects_five_objects   0.180000  0.212000
tracking_shuffled_objects_seven_objects  0.164000  0.136000
tracking_shuffled_objects_three_objects  0.268000  0.312000
web_of_lies                              0.520000  0.524000
word_sorting                             0.068000  0.016000

## Observations
- Flan improves performance a lot, over t5. 
- Flan didn't give t5 CoT capability. Across all t5 series, CoT performs worse than direct prompt. 
- CoT performs better over direct prompt only with 540b models.

Since CoT is no show in t5, no point to do failure case analysis. Nothing to analyze for direction prompt. And you would only find the same mistakes as in CoT papaer for CoT prompted failure cases. 